# Binomial Asset Pricing model
The binomial trees will be treated as an etwork with nodes (i,j) with
* i representing the timesteps
* j representng the number of ordered price outcome, respectively from the lowest (bottom of the tree) to the highest

Par définition, stock tree may be represented as above
> $S(i,j) = S_0u^jd^{i-j}$ with $S_0$ being the initial stock price

The same way, $C_i,_j$ represents the contract price at each (i,j) node with $C_N,_j$ is the final payoff function. The case studied in this notebook is European Call price, so
 > $C_N,_j = max(S_N,_j-K, 0)$

In [5]:
try:
    import numpy as np
    import pandas as pd
    
    from functools import wraps
    from time import time
except ImportError:
    print('Missing Library')

In [21]:
def timing(f):
    @wraps(f)
    def wrap(*args, **kwargs):
        ts = time()
        result = f(*args, **kwargs)
        te = time()
        print('____________________________________________________________')
        print(f'func: {f.__name__} args: [{args},  {kwargs}] took: {te-ts} ')
        return result
    return wrap

S0 = 100       # Stock Initial
K = 100        # Strike Price
r = 0.06       # Annual Risk Free Rate
N, T = 3, 1    # Number of Steps, Time to Maturity (in years)
u = 1.1        # Factor Up 
d = 1/u        # Factor Down
opttype = 'C'  # Option Type 'C'all or 'P'ut

## Simple Slow Binomial Pricing Model 

In [13]:
@timing
def BTSlow(K, T, S0, r, N, u, d, opttype='C'):
    # precompute consts
    dt = T/N
    q = (np.exp(r*dt) - d)/(u - d)
    disc = np.exp(-r*dt)
    
    # initialize assets prices at maturity - Time step N
    S = np.zeros(N+1)
    S[0] = S0 * d ** N
    for j in range(1, N+1):
        S[j] = u * S[j-1]/d
        
    # initialize option values at maturity
    C = np.zeros(N+1)
    for j in range(0, N+1):
        C[j] = max(0, S[j] - K)
    
    # Step backwards through tree
    for i in np.arange(N, 0, -1):
        for j in range(0, i):
            C[j] = disc * (q * C[j+1] + (1-q)*C[j])
    
    return C[0]

# Testing the Slow Pricing Model
BTSlow(K, T, S0, r, N, u, d)

func: BTSlow args: [(100, 1, 100, 0.06, 3, 1.1, 0.9090909090909091), {}] took: 0.00025725364685058594 


10.145735799928817

## Simple Fast Binomial Pricing Model 
This one will be better as the number of nodes increases. In fact, the function will call for vectors rather than looping to find the values

In [14]:
@timing
def BTFast(K, T, S0, r, N, u, d, opttype='C'):
    
    dt = T/N
    q = (np.exp(r*dt) - d)/(u - d)
    disc = np.exp(-r*dt)
    
    # Option Values 
    downP = d**(np.arange(N, -1, -1))
    upP = u **(np.arange(0, N+1, 1))
    C = S0 * downP * upP
    C = np.maximum(C - K, np.zeros(N+1))
    
    # Finding C values
    for i in np.arange(N, 0, -1):
        C = disc * (q * C[1:i+1] + (1 - q) * C[0:i])
    
    return C[0]

# Testing the Slow Pricing Model
BTFast(K, T, S0, r, N, u, d)

func: BTFast args: [(100, 1, 100, 0.06, 3, 1.1, 0.9090909090909091), {}] took: 0.0021080970764160156 


10.145735799928826

## Comparison of Fast vs Slow
Comparison of computation time of both pricing models over the number of years N

In [28]:
for N in [3, 5, 10, 15, 50, 77]:
    print(115*'_')
    BTSlow(K, T, S0, r, N, u, d, opttype='C')
    BTFast(K, T, S0, r, N, u, d, opttype='C')

___________________________________________________________________________________________________________________
func: BTSlow args: [(100, 1, 100, 0.06, 3, 1.1, 0.9090909090909091), {'opttype': 'C'}] took: 8.249282836914062e-05 
func: BTFast args: [(100, 1, 100, 0.06, 3, 1.1, 0.9090909090909091), {'opttype': 'C'}] took: 0.00011992454528808594 
___________________________________________________________________________________________________________________
func: BTSlow args: [(100, 1, 100, 0.06, 5, 1.1, 0.9090909090909091), {'opttype': 'C'}] took: 3.743171691894531e-05 
func: BTFast args: [(100, 1, 100, 0.06, 5, 1.1, 0.9090909090909091), {'opttype': 'C'}] took: 7.605552673339844e-05 
___________________________________________________________________________________________________________________
func: BTSlow args: [(100, 1, 100, 0.06, 10, 1.1, 0.9090909090909091), {'opttype': 'C'}] took: 6.437301635742188e-05 
func: BTFast args: [(100, 1, 100, 0.06, 10, 1.1, 0.9090909090909091), 